In [1]:
source = "CombineSignals";

In [2]:
using NBInclude
import XGBoost
@nbinclude("Alpha.ipynb");

In [3]:
@nbinclude("../TrainingAlphas/XGBoostFeatures.ipynb");

In [4]:
# used for XGBoostFeatures
function read_params(alpha)
    params = load("../../data/alphas/$alpha/params.jld2")
    if alpha == "UserItemBiases"
        μ = mean(read_recommendee_alpha("UserItemBiases").rating - params["a"])
        push!(params["u"], μ)
    end
    params
end;

In [5]:
params = read_params(source);

In [6]:
X = get_augmented_indep("recommendee_inference", params["alphas"], params["β"]);

Progress: 100%|█████████████████████████████████████████| Time: 0:00:00
Progress: 100%|███████████████████████████| Time: 0:00:00 ( 0.23 μs/it)it)
Progress: 100%|███████████████████████████| Time: 0:00:00 ( 0.18 μs/it)
Progress: 100%|███████████████████████████| Time: 0:00:02 ( 0.57 μs/it)
Progress: 100%|███████████████████████████| Time: 0:00:01 ( 0.39 μs/it)
Progress: 100%|███████████████████████████| Time: 0:00:02 ( 0.54 μs/it)
Progress: 100%|███████████████████████████| Time: 0:00:03 ( 0.88 μs/it)
Progress: 100%|███████████████████████████| Time: 0:00:00 (95.26 μs/it)
[ Debug: 20220117 02:16:12 nsfw categories: String7["white"]
Progress: 100%|███████████████████████████| Time: 0:00:00 ( 0.24 ms/it)
[ Debug: 20220117 02:16:13 medium categories: String7["movie", "music", "ona", "ova", "special", "tv", "unknown"]
[ Debug: 20220117 02:16:14 source categories: String15["4_koma_manga", "NaN", "book", "card_game", "digital_manga", "game", "light_novel", "manga", "mixed_media", "music", "n

In [7]:
bst = XGBoost.Booster(model_file = params["bst"])
boost_preds = XGBoost.predict(bst, X)
predictions =
    get_indep("recommendee_inference", params["alphas"]) * params["β"] + boost_preds;

In [8]:
@debug "average bst magnitude: $(mean(abs.(boost_preds))), $(sqrt(mean(boost_preds .^ 2)))"

[ Debug: 20220117 02:17:17 average bst magnitude: 0.15520689, 0.20645452


In [9]:
model(items) = predictions[items]
write_recommendee_alpha(model);

## Error model

In [10]:
error_bst = XGBoost.Booster(model_file = params["error_bst"])
error_preds = sqrt.(clamp.(XGBoost.predict(error_bst, hcat(X, boost_preds)), 0, Inf));

In [12]:
model(items) = error_preds[items]
write_recommendee_alpha(model, outdir = "PredictedErrors");